<a href="https://colab.research.google.com/github/ahmadabousalem/sentinel-threat-map/blob/main/maps/risk_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install feedparser pandas --quiet

import feedparser
import pandas as pd

# Define cities you want to monitor
cities = ["Beirut", "Baghdad", "Damascus", "Gaza"]

# RSS news feeds
rss_feeds = [
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml",
    "https://www.aljazeera.com/xml/rss/all.xml",
    "https://rss.dw.com/rdf/rss-en-all"
]

# Fetch and filter articles
def fetch_articles():
    articles = []
    for feed_url in rss_feeds:
        feed = feedparser.parse(feed_url)
        for entry in feed.entries:
            for city in cities:
                if city.lower() in entry.title.lower() or city.lower() in entry.summary.lower():
                    articles.append({
                        "City": city,
                        "Title": entry.title,
                        "Summary": entry.summary[:200],
                        "Published": entry.published if "published" in entry else "N/A",
                        "Link": entry.link
                    })
    return pd.DataFrame(articles)

# Fetch and display
df = fetch_articles()
df.head(10)  # Show first 10 articles


,City,Title,Summary,Published,Link
0,Gaza,Netanyahu to propose full reoccupation of Gaza...,The plan will involve a complete takeover of G...,"Tue, 05 Aug 2025 15:06:52 GMT",https://www.bbc.com/news/articles/cpqv2qjg5vvo...
1,Gaza,"He is a human skeleton, Gaza hostage's brother...",Ilay David says the Hamas video showing his br...,"Mon, 04 Aug 2025 17:05:21 GMT",https://www.bbc.com/news/articles/cewyk4ezeedo...
2,Gaza,Hundreds of Israeli ex-officials appeal to Tru...,It comes amid reports that PM Netanyahu is pus...,"Mon, 04 Aug 2025 13:01:22 GMT",https://www.bbc.com/news/articles/crkznje8nz8o...
3,Gaza,World leaders condemn videos of emaciated Isra...,Hamas said it would allow Red Cross access if ...,"Sun, 03 Aug 2025 21:15:58 GMT",https://www.bbc.com/news/articles/crr2dwn7q40o...
4,Gaza,Aid group says worker killed by Israeli milita...,The Palestine Red Crescent Society accuses Isr...,"Sun, 03 Aug 2025 21:43:27 GMT",https://www.bbc.com/news/articles/cx2x5eyl676o...
5,Gaza,Trump envoy visits controversial Gaza aid dist...,The visit with the IDF follows near daily repo...,"Fri, 01 Aug 2025 18:14:15 GMT",https://www.bbc.com/news/articles/c8x5jnl5eygo...
6,Gaza,France suspends Gaza evacuations after student...,"The Palestinian student, who arrived in France...","Fri, 01 Aug 2025 17:15:39 GMT",https://www.bbc.com/news/articles/c5yl7n42325o...
7,Gaza,Gazan boy first to be treated in UK for war in...,"Majd, 15, was almost killed by an Israeli tank...","Thu, 31 Jul 2025 18:07:39 GMT",https://www.bbc.com/news/articles/cvgn6979n59o...
8,Gaza,US allies break with Trump to force diplomatic...,As Canada joins France and the UK with plans t...,"Thu, 31 Jul 2025 01:44:50 GMT",https://www.bbc.com/news/articles/cly6rl6158no...
9,Gaza,Gaza aid site offered a 'women only' day. It d...,Families say mothers Mary Sheikh al-Eid and Kh...,"Sat, 26 Jul 2025 18:05:36 GMT",https://www.bbc.com/news/articles/c74z4gy5g31o...


In [21]:
pip install feedparser pandas folium --quiet


In [22]:
import feedparser
import pandas as pd
import folium


In [23]:
city_coords = {
    "Beirut": (33.8938, 35.5018),
    "Baghdad": (33.3152, 44.3661),
    "Damascus": (33.5138, 36.2765),
    "Jerusalem": (31.7683, 35.2137)
}


In [24]:
rss_feeds = [
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml"
]


In [25]:
def fetch_risk_data(cities, feeds):
    counts = {city: 0 for city in cities}
    for feed_url in feeds:
        feed = feedparser.parse(feed_url)
        for entry in feed.entries:
            title = entry.title.lower()
            for city in cities:
                if city.lower() in title:
                    counts[city] += 1
    return counts


In [18]:
def get_risk_level(count):
    if count > 5:
        return "High"
    elif count > 2:
        return "Medium"
    elif count >= 0:
        return "Low"
    else:
        return "No data"


In [19]:
risk_data = fetch_risk_data(city_coords.keys(), rss_feeds)
print("Risk data:", risk_data)


Risk data: {'Beirut': 0, 'Baghdad': 0, 'Damascus': 0, 'Jerusalem': 2}


In [20]:
m = folium.Map(location=[33.5, 36.5], zoom_start=6)

for city, coords in city_coords.items():
    count = risk_data.get(city, 0)
    risk = get_risk_level(count)
    color = {"High": "red", "Medium": "orange", "Low": "green", "No data": "gray"}.get(risk, "gray")
    folium.Marker(
        location=coords,
        popup=f"<b>{city}</b><br>Articles: {count}<br>Risk: {risk}",
        tooltip=f"{city}: {risk}",
        icon=folium.Icon(color=color)
    ).add_to(m)

m